## Label Frames

Tool for viewing each frame in a directory and applying a label to it. This is assumed to be a single number (such in the case of labelling an instrument's reading) but could be expanded to apply multiple values.


In [1]:
import os
import cv2
import csv

## Read / Write to CSV

In [2]:
def get_csv_contents(csv_path):

    csv_exists = os.path.exists(csv_path)

    lines = []

    if csv_exists:
        with open(csv_path, mode='r', newline='') as csv_file:
            for index, line in enumerate(csv_file):

                if index == 0:
                    continue

                frame = line.split(",")[0]
                label = line.split(",")[1]

                label = label[:-2]

                lines.append({"frame": frame, "label": label})

    return lines


LABEL_FIELDS = ['image_path', 'airspeed']

def save_csv(filename, labels):

    with open(filename, mode='w', newline='') as csvfile:

        writer = csv.DictWriter(csvfile, fieldnames=LABEL_FIELDS)

        writer.writeheader()

        for line in labels:
            writer.writerow({LABEL_FIELDS[0]: line['frame'], LABEL_FIELDS[1]: line['label']})

## Make Label CSV match files in directory

This opens the labels CSV file, then adds any images that are in the image_directory but not in the label frames to the array. Labels are initialized to an empty string

In [3]:
def get_labels(image_directory, label_path):

    image_files = [f for f in os.listdir(image_directory) if f.endswith('.jpg')]

    image_files.sort()

    labels = get_csv_contents(label_path)

    labelled_images = [im['frame'] for im in labels]

    for image in image_files:
        if image not in labelled_images:
            labels.append({"frame": image, "label": ""})

    return labels

## Display Frames

This has all of the application logic. It could probably be cleaned up a bit

In [5]:
# Hacky means of making sure we don't go into infinite loop
def exit_application():
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.waitKey(1)
    cv2.waitKey(1)

LABEL_INCREMENT = 10

def display_frames(image_directory, output_file, labels):

    all_images = [im['frame'] for im in labels]
    current_index = -1
    current_label = 0

    while True:

        if current_index == -1:
            image_path = '../media/utility_frames/instructions.jpg'
        else:
            image_path = os.path.join(image_directory, all_images[current_index])

        if not os.path.exists(image_path):
            image_path = '../media/utility_frames/frame_not_found.jpg'

        existing_label = labels[current_index]['label']

    
        # Read the image
        image = cv2.imread(image_path)

        if current_index >= 0:
            # Overlay text information onto the frame
            cv2.putText(image, f'F: {current_index + 1} / {len(all_images)}', (10, 50), cv2.FONT_HERSHEY_PLAIN, 1.0, (0, 255, 0), 2)
            cv2.putText(image, f'E: {existing_label}', (10, 10), cv2.FONT_HERSHEY_PLAIN, 1.0, (0, 255, 0), 2)
            cv2.putText(image, f'R: {current_label}', (10, 30), cv2.FONT_HERSHEY_PLAIN, 1.0, (0, 255, 0), 2)
            
        cv2.imshow(f'Frame Labeller', image)

        # Wait for user input
        key = cv2.waitKey(0)

        # Spacebar - Save current label and move to next image
        if key == ord(' '):  

            if current_index >= 0:
                labels[current_index]['label'] = current_label

                save_csv(output_file, labels)
                
            if current_index < len(all_images) - 1:
                current_index += 1
        # 'q' - Exit the application
        elif key == ord('q'):
            exit_application()
            break
        # Up Arrow - Increase the label
        elif key == 0:
            current_label += LABEL_INCREMENT
        # Down Arrow - Decrease the label
        elif key == 1:
            current_label -= LABEL_INCREMENT
        # Right Arrow - Skip to next frame without saving
        elif key == 3:
            if current_index < len(all_images) - 1:
                current_index += 1
        elif key == 2:
            if current_index > -1:
                current_index -= 1

    exit_application()

## Start Frame Labeller

In [6]:
image_directory = '../media/night_fixed_A10C_validation_airspeed/'
label_file_path = '../image_labels/night_airspeed_validation_labels.csv'

labels = get_labels(image_directory, label_file_path)

cv2.startWindowThread()

display_frames(image_directory, label_file_path, labels)